# Model V3

In [5]:
import zipfile
from pathlib import Path

import cv2 as cv
import numpy as np
import onnx
import matplotlib.pyplot as plt
import tensorflow as tf
import tf2onnx
import tqdm
from keras import layers
from keras import models
from keras import callbacks
from keras import losses
from keras import metrics
from keras import optimizers

train_dirs = list((Path.cwd() / 'data' / 'data_0').glob('**/train/'))
test_dirs = list((Path.cwd() / 'data' / 'data_0').glob('**/test/'))

def dataset_file_get_timestamp(file: Path):
    return int(file.stem.split('_')[0])

def dataset_file_get_steering_angle(file: Path):
    return float(file.stem.split('_')[1].replace('-', '.'))

def load_data(directory: Path):
    image_paths = []
    steering_angles = []
    # load image paths from the directory
    for file_path in directory.glob('*.png'):
        image_paths.append(file_path)

    # sort images by the timestamp in their filenames
    image_paths.sort(key=dataset_file_get_timestamp)

    # Extract steering angles from filenames
    for path in image_paths:
        steering_angles.append(dataset_file_get_steering_angle(path))
        
    return image_paths, steering_angles

train_img = []
train_steer = []
test_img = []
test_steer = []

for test_dir in test_dirs:
    test_img_, test_steer_ = load_data(test_dir)
    test_img += test_img_
    test_steer += test_steer_

for train_dir in train_dirs:
    train_img_, train_steer_ = load_data(train_dir)
    train_img += train_img_
    train_steer += train_steer_

BATCH_SIZE = 50
RAND_SEED = 42

SHUFFLE_BUFFER_DIVIDER = 2
PREFETCH_BUFFER_SIZE = tf.data.experimental.AUTOTUNE

def process_image(image_path):
    img = cv.imread(image_path) 
    hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    mask = cv.inRange(hsv, (36, 0, 0), (70, 255,255))
    array = np.array(mask, dtype=np.float32)
    return array

def dataset_pipeline(features, labels, shuffle):
    global BATCH_SIZE, RAND_SEED, SHUFFLE_BUFFER_DIVIDER, PREFETCH_BUFFER_SIZE

    dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    dataset = dataset.repeat()
    if shuffle:
        print("Shuffling dataset")
        dataset = dataset.shuffle(
            buffer_size=len(features) // SHUFFLE_BUFFER_DIVIDER,
            seed=RAND_SEED
        )
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(buffer_size=PREFETCH_BUFFER_SIZE)
    return dataset

train_data = []
test_data = []

tqdm.tqdm.write("Loading training data")
# load the images and steering angles into numpy arrays
for path in tqdm.tqdm(train_img):
    train_data.append(process_image(path))

tqdm.tqdm.write("Loading test data")
# loading the testing data
for path in tqdm.tqdm(test_img):
    test_data.append(process_image(path))

## create a dataset from the training data by using the dataset pipeline
print("Creating training dataset")
train_ds = dataset_pipeline(train_data, train_steer, shuffle=True)

## create a dataset from the testing data by using the dataset pipeline
print("Creating testing dataset")
test_ds = dataset_pipeline(test_data, test_steer, shuffle=False)

print("Done!")

model = models.Sequential([
    layers.InputLayer(shape=(66, 100, 1)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='linear')
])

model.summary()
optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse')

## Training parameters
# Specify the various parameters for training
BATCH_DIVIDER = 1
EPOCHS = 25
LEARNING_RATE = 0.001
OPTIMIZER = optimizers.Adam(learning_rate=LEARNING_RATE)
LOSS = losses.MeanSquaredError()
METRICS = metrics.RootMeanSquaredError()

fit_params = {
    'epochs': EPOCHS
}

model.compile(optimizer=OPTIMIZER, loss=LOSS)

hist = model.fit(train_ds,
                 steps_per_epoch=len(train_img) // BATCH_SIZE // BATCH_DIVIDER,
                 validation_data=test_ds,
                 validation_steps=len(test_img) // BATCH_SIZE // BATCH_DIVIDER,
                 **fit_params)

eval_out = model.evaluate(test_ds, steps=50)

Loading training data


100%|██████████| 10000/10000 [00:00<00:00, 12401.94it/s]


Loading test data


100%|██████████| 6000/6000 [00:00<00:00, 12568.99it/s]


Creating training dataset
Shuffling dataset
Creating testing dataset
Done!


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 64, 98, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 32, 49, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 30, 47, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 15, 23, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 44160)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │     2,826,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,902,913 (11.07 MB)

 Trainable params: 2,902,913 (11.07 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 1226.6481 - val_loss: 31.4753
Epoch 2/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 15.0155 - val_loss: 23.2282
Epoch 3/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 9.9441 - val_loss: 18.6572
Epoch 4/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 6.1426 - val_loss: 13.8148
Epoch 5/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 3.8371 - val_loss: 15.3333
Epoch 6/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 3.1964 - val_loss: 13.1475
Epoch 7/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 2.7412 - val_loss: 15.7940
Epoch 8/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 3.9032 - val_loss: 16.6615
Epoch 9/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 6.6749 - val_loss: 11.4572
Epoch 10/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 2.1893 - val_loss: 10.8351
Epoch 11/25
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 2.5252 - val_loss: 10.9119
Epoch 12/25
200/200 ━━━━━━━━━━━━━━━

In [7]:


filename = 'v3'
print(f"Saving model as {filename}.h5")
model.save(f'{filename}.keras')
model.save(f'{filename}.h5')
model.output_names = ['output']
input_signature = [tf.TensorSpec([None, 66, 100, 1], tf.float32, name='input')]
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=input_signature, opset=13)
onnx.save(onnx_model, filename + ".onnx")

Saving model as v3.h5


I0000 00:00:1743671609.818824   53584 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1743671609.819751   53584 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2025-04-03 02:13:29.819841: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
I0000 00:00:1743671609.820062   53584 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1743671609.820087   53584 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1743671609.820100   53584 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been bui